In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd

/content


In [3]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [4]:
%cd Chatbot

/content/drive/MyDrive/Chatbot


In [5]:
!ls

config.json  Llama_chat.ipynb  main.ipynb  requirements.txt  Untitled11.ipynb  vision.ipynb


In [6]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 91.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [7]:
import json
import torch
from huggingface_hub import login
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          pipeline,
                          BlipForQuestionAnswering,
                          AutoProcessor)
from PIL import Image
import os

In [9]:
# Load configuration
config_data = json.load(open("config.json"))
HF_TOKEN = config_data["HF_TOKEN"]
login(HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
# Set the model name for the text-based chatbot (LLaMA)
model_name = "meta-llama/Llama-2-13b-chat-hf"

In [11]:
# Config for loading the LLaMA model in 4-bit precision
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [12]:
# Load the tokenizer and model for the text-based chatbot
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=HF_TOKEN)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:796: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [13]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             torch_dtype=torch.float16,
                                             quantization_config=bnb_config,
                                             token=HF_TOKEN)

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [14]:
 # Load the pipeline for text generation
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

In [15]:
# Load the pretrained BLIP model and processor for VQA
vqa_model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
vqa_processor = AutoProcessor.from_pretrained("Salesforce/blip-vqa-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
# Function to perform question answering on an image (from Code B)
def vqa(image_path, question):
    # Open the image using PIL
    image = Image.open(image_path)

    # Process the image and question using the processor
    inputs = vqa_processor(image, question, return_tensors="pt")

    # Generate the answer using the BLIP model
    out = vqa_model.generate(**inputs)

    # Decode the generated output (token IDs) into a readable answer
    answer = vqa_processor.decode(out[0], skip_special_tokens=True)

    return answer

In [17]:
# Combined chatbot function (Text-based + Image-based)
def chat():
    print("Chatbot: Hello! How can I assist you today?")

    while True:
        user_input = input("You: ")

        # If user wants to exit the chatbot
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Chatbot: Goodbye!")
            break

        # Check if the user input indicates an image-based question
        if user_input.startswith("image:"):
            try:
                image_path, question = user_input.split(" ", 1)
                image_path = image_path.split("image:")[1]  # Extract the image path

                # Check if the image file exists
                if not os.path.exists(image_path):
                    print("Chatbot: The image path provided does not exist.")
                    continue

                # Perform visual question answering using BLIP model
                answer = vqa(image_path, question)
                print(f"Chatbot: {answer.strip()}\n")

            except Exception as e:
                print(f"Chatbot: Error processing image-based question: {str(e)}\n")

        else:
            # For text-based conversation, use the LLaMA model
            try:
                response = text_generator(user_input, max_new_tokens=200)[0]['generated_text']
                print(f"Chatbot: {response.strip()}\n")

            except Exception as e:
                print(f"Chatbot: Error generating text-based response: {str(e)}\n")


In [22]:
if __name__ == "__main__":
    chat()

Chatbot: Hello! How can I assist you today?
You: /content/drive/MyDrive/Chatbot/images.jpeg Who is this man in the image?
Chatbot: /content/drive/MyDrive/Chatbot/images.jpeg Who is this man in the image?

The man in the image is Elon Musk.

You: bye
Chatbot: Goodbye!
